# Camera Calibration from Scratch

---
This repository is mainly focused on details of camera calibration implementation in OpenCV. It include both [narrow field of view](https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html) and [fisheye](https://docs.opencv.org/3.4/db/d58/group__calib3d__fisheye.html) monocular calibration. OpenCV's camera calibration is mainly based on this [Matlab toolbox](http://www.vision.caltech.edu/bouguetj/calib_doc/index.html#links) implemented by Jean-Yves Bouguet and that is what I used as a reference to understand how the code is working. Also, I have used a few other references that are all listed at the end of this document.


## Camera Calibration Steps

1. Construct checkerboard 3D points (object points) according to checkerboard shape and size 
2. Detect checkerboard corners (image points)
3. Estimate initial values for intrinsic parameters
    1. Remove effect of distortion. This is only needed for fisheye model.
    2. Estimate plane to plane homohraphy from object points to image points.
    3. Estimate camera instrinsics.
4. Use estimated intrinsic parameters to estimate extrinsic parameters for eacg image
    1. Normalize images
    2. Estimate plane to plane homography
    3. Estimate extrinsic values from homography matrix.
5. Use all of the initial intrinsic and extrinsic values in a non-linear optimization and estimate optimal intrinsic and extrinsic values.

## Calculating Homography

### Derivation of Homogeneous Equation (Direct Linear Transformation)

Given a set of $M$ 2D<->2D point correspondances in homogenous coordinate frame where $M \geq 4$, homography matrix $H$, where $H$ is given by the equation $\mathbf{x_i'} = \mathbf{Hx_i}$ ,can be calculated as follows (Ref. 4, Multiple View Geometry, Section 4.1):

\begin{aligned}
\mathbf{x_i} = \begin{pmatrix}x_i\\y_i\\w_i\end{pmatrix}, \,\,
\mathbf{x_i'} = \begin{pmatrix}x_i'\\y_i'\\w_i'\end{pmatrix}, \,\,
\mathbf{H}=
\begin{bmatrix}
h_1 & h_2 & h_3 \\ h_4 & h_5 & h_6 \\ h_7 & h_8 & h_9
\end{bmatrix} = 
\begin{bmatrix} 
\mathbf{{h^1}^T} \\ \mathbf{{h^2}^T} \\ \mathbf{{h^3}^T}
\end{bmatrix}\\
\mathbf{Hx_i} =
\begin{pmatrix}
\mathbf{{h^1}^Tx_i} \\ \mathbf{{h^1}^Tx_i} \\ \mathbf{{h^3}^Tx_i}
\end{pmatrix}
\end{aligned}

Because this equation involves homogeneous vectors, $\mathbf{x_i'}$ and $\mathbf{Hx_i}$ are not equal, but they have the same dirrection. The equation can be expressed in terms of vector cross product $\mathbf{x_i' \times Hx_i} = \mathbf0$. This gives a set of three equations in the entires of \mathbf{H}, which can be written in the form

\begin{aligned}
\begin{bmatrix}
\mathbf{0^T} & -w_i' \mathbf{x_i^T} & y_i' \mathbf{x_i^T}\\
w_i' \mathbf{x_i^T} & \mathbf{0^T} & -x_i' \mathbf{x_i^T}\\
-y_i' \mathbf{x_i^T} & x_i' \mathbf{x_i^T} & \mathbf{0^T}
\end{bmatrix}
\begin{pmatrix}
\mathbf{h^1}\\
\mathbf{h^2}\\
\mathbf{h^3}
\end{pmatrix} = \mathbf{0}
\end{aligned}

This equation has the form $\mathbf{A_ih} = \mathbf{0}$, where $\mathbf{A_i}$ is a $3 \times 9$ matrix and $\mathbf{h}$ is a 9-vector made up of the entires of the matrix $\mathbf{H}$,

\begin{aligned}
\mathbf{h} = \begin{pmatrix}
\mathbf{h^1}\\\mathbf{h^2}\\\mathbf{h^2}
\end{pmatrix}
\end{aligned}

Although there are three equations, two of them or linearly independent (third row can be calculated up to scale from the sum of $x_i'$ times the first row and $y_i'$ times the second row). It is usual to omit the third equation, then the set of equations becomes 

\begin{aligned}
\begin{bmatrix}
\mathbf{0^T} & -w_i' \mathbf{x_i^T} & y_i' \mathbf{x_i^T}\\
w_i' \mathbf{x_i^T} & \mathbf{0^T} & -x_i' \mathbf{x_i^T}\\
\end{bmatrix}
\begin{pmatrix}
\mathbf{h^1}\\
\mathbf{h^2}\\
\mathbf{h^3}
\end{pmatrix} = \mathbf{0}
\end{aligned}


For each set of corresponding points we construct two rows for the matruix $\mathbf{A}$ and finally solve homogenous equation $\mathbf{Ah} = \mathbf{0}$ using Singular Value Decomposition (SVD) and then reformat vector $\mathbf{h}$ to matrix $\mathbf{H}$.


### Normalizing Transformations

Because without normalization the scale of image points $x'$ and $y'$ are much larger than $w'$ (a few hunvreds vs. 1), it is necessary to normalize the points before constructing the matrix $\mathbf{A}$. The effect of this normalization is removed from the final homography matrix by applying the reverse of this normalization. Please see **Algorithm 4.2** in Multiple View Geometry (Ref. 4).

Normalization steps:

1. Translate the points so their centroid is at the origin.
2. Scale the points, so their average distance from the origin is equal to $\sqrt2$.
3. Apply this transformation to each of the images independently.

For a set of $M$ points $\mathbf{x_i}$ in one image that are scaled to $w_i=1$, transformation matrix can be constructed as follows:

\begin{aligned}
T = \begin{bmatrix}
1/\alpha_x & 0 & -t_x/\alpha_x\\
0 & 1/\alpha_y & -t_y/\alpha_y\\
0 & 0 & 1
\end{bmatrix}
\end{aligned}

where

\begin{aligned}
t_x = \frac{1}{M}\sum_{i=1}^{M}{x_i}\\
t_y = \frac{1}{M}\sum_{i=1}^{M}{y_i}\\
\alpha_x = \frac{1}{M}\sum_{i=1}^{M}{\left|x_i - t_x\right|}\\
\alpha_y = \frac{1}{M}\sum_{i=1}^{M}{\left|y_i - t_y\right|}
\end{aligned}







### Normalized Direct Linear Transformation Algorithm

1. For two sets of points $x_i$ and $x_i'$ calulate transformations $T$ and $T'$.
2. Normalize each sets of points as $\tilde{x_i} = Tx_i$ and $\tilde{x_i'} = Tx_i'$
3. Calculate homography matrix $\tilde{H}$ using direct linear transformation algorithm where $\tilde{x_i'} = \tilde{H}\tilde{x_i}$.
4. Removing the effect of normalization: $H = {T'}^{-1}\tilde{H}T$
5. Normalize $H$ by diving all of the elemnts by $h_9$.


### Nonlinear Optimization

To further improve the accuracy of $\mathbf{H}$, we use the homography matrix estimated in previous section as initial values and refine the matrix through a Gauss-Newton non-linear least squares optimization as follows:

Let $\beta$ be the parameter vector:

\begin{align}
\mathbf{\beta^T} = \begin{pmatrix} h_1 & h_2 & h_3 & h_4 & h_5 & h_6 & h_7 & h_8 \end{pmatrix}\\
\end{align}

### Rodrigues

This function converts an axis-angle vector $V$  to a rotation matrix $R$

#### Calculating the rotation matrix
\begin{align}
R = Rodrigues(\mathbf{v})\\
\\
\mathbf{v} = 
\begin{pmatrix}
v_x\\
v_y\\
v_z\\
\end{pmatrix} \\
\\
\theta = \sqrt{v_x^2+ v_y^2+v_z^2}\\
\mathbf{u}=\frac{1}{\theta}\mathbf{v}=
\begin{pmatrix}
u_x\\
u_y\\
u_z\\
\end{pmatrix} \\
\\
u_x^2+ u_y^2+u_z^2 =1\\
\\
\begin{bmatrix}\mathbf{u} \end{bmatrix}_\times = \begin{pmatrix}
 0 & -u_z & u_y\\
 u_z & 0 & -u_x\\
 -u_y & u_x & 0
 \end{pmatrix} \\
 \\
\mathbf{u}\otimes \mathbf{u} = \mathbf{u}\mathbf{u}^T = 
\begin{pmatrix}
 u_x^2 & u_xu_y & u_xu_z\\
 u_yu_x & u_y^2 & u_yu_z\\
 u_zu_x & u_zu_y & u_z^2
 \end{pmatrix}\\
 \\
R=(cos\theta)I + (sin\theta)\begin{bmatrix}\mathbf{u} \end{bmatrix}_\times + (1-cos\theta)(\mathbf{u}\otimes \mathbf{u})\\
\end{align}

#### Calculating the jacobian matrix

Calculate Jacobian A where inependent variables are $v_x, v_y, v_z$


\begin{align}
A = \begin{pmatrix}
\frac{\partial v_x}{\partial v_x} & \frac{\partial v_x}{\partial v_y} & \frac{\partial v_x}{\partial v_z}\\
\frac{\partial v_y}{\partial v_x} & \frac{\partial v_y}{\partial v_y} & \frac{\partial v_y}{\partial v_z}\\
\frac{\partial v_z}{\partial v_x} & \frac{\partial v_z}{\partial v_y} & \frac{\partial v_z}{\partial v_z}\\
\frac{\partial \theta}{\partial v_x} & \frac{\partial \theta}{\partial v_y} & \frac{\partial \theta}{\partial v_z}\\
\end{pmatrix} = 
\begin{pmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1\\
\frac{v_x}{\theta} & \frac{v_y}{\theta} & \frac{v_z}{\theta}\\
\end{pmatrix}
\end{align}


Calculate Jacobian B where inependent variables are $v_x, v_y, v_z, \theta$


\begin{align}
B = \begin{pmatrix}
\frac{\partial u_x}{\partial v_x} & \frac{\partial u_x}{\partial v_y} & \frac{\partial u_x}{\partial v_z} & \frac{\partial u_x}{\partial \theta}\\
\frac{\partial u_y}{\partial v_x} & \frac{\partial u_y}{\partial v_y} & \frac{\partial u_y}{\partial v_z} & \frac{\partial u_y}{\partial \theta}\\
\frac{\partial u_z}{\partial v_x} & \frac{\partial u_z}{\partial v_y} & \frac{\partial u_z}{\partial v_z} & \frac{\partial u_z}{\partial \theta}\\
\frac{\partial \theta}{\partial v_x} & \frac{\partial \theta}{\partial v_y} & \frac{\partial \theta}{\partial v_z} & \frac{\partial \theta}{\partial \theta}\\
\end{pmatrix} = 
\begin{pmatrix}
\frac{1}{\theta} & 0 & 0 & \frac{-v_x}{\theta^2}\\
0 & \frac{1}{\theta} & 0 & \frac{-v_y}{\theta^2}\\
0 & 0 & \frac{1}{\theta} & \frac{-v_z}{\theta^2}\\
0 & 0 & 0 & 1\\
\end{pmatrix}
\end{align}

\begin{align}
R =\begin{pmatrix}
r_{11}\\
r_{12}\\
r_{13}\\
r_{21}\\
r_{22}\\
r_{23}\\
r_{31}\\
r_{32}\\
r_{33}
\end{pmatrix}=
(cos\theta)
\begin{pmatrix}
1\\
0\\
0\\
0\\
1\\
0\\
0\\
0\\
1
\end{pmatrix} +
(sin\theta)
\begin{pmatrix}
 0\\ -u_z\\ u_y\\ u_z\\ 0 \\\ -u_x\\-u_y \\ u_x \\ 0
 \end{pmatrix} +
(1-cos\theta)
\begin{pmatrix}
 u_x^2 \\ u_xu_y \\ u_xu_z\\
 u_yu_x \\ u_y^2 \\ u_yu_z\\
 u_zu_x \\ u_zu_y \\ u_z^2
 \end{pmatrix}
\end{align}

\begin{align}
\frac{\partial R}{\partial u_x}=
(sin\theta)
\begin{pmatrix}
0\\ 0\\ 0\\ 0\\ 0\\ -1\\ 0\\ 1\\ 0\\
\end{pmatrix} +
(1-cos\theta)
\begin{pmatrix}
2u_x\\u_y\\u_z\\u_y\\0\\0\\u_z\\0\\0
\end{pmatrix}
\end{align}

\begin{align}
\frac{\partial R}{\partial u_y}=
(sin\theta)
\begin{pmatrix}
0\\ 0\\ 1\\ 0\\ 0\\ 0\\ -1\\ 0\\ 0\\
\end{pmatrix} +
(1-cos\theta)
\begin{pmatrix}
0\\u_x\\0\\u_x\\2u_y\\u_z\\0\\u_z\\0
\end{pmatrix}
\end{align}


\begin{align}
\frac{\partial R}{\partial u_z}=
(sin\theta)
\begin{pmatrix}
0\\ -1\\ 0\\ 1\\ 0\\ 0\\ 0\\ 0\\ 0\\
\end{pmatrix} +
(1-cos\theta)
\begin{pmatrix}
0\\0\\u_x\\0\\0\\u_y\\u_x\\u_y\\2u_z
\end{pmatrix}
\end{align}


\begin{align}
\frac{\partial R}{\partial \theta}=
-(sin\theta)
\begin{pmatrix}
1\\
0\\
0\\
0\\
1\\
0\\
0\\
0\\
1
\end{pmatrix} +
(cos\theta)
\begin{pmatrix}
 0\\ -u_z\\ u_y\\ u_z\\ 0 \\\ -u_x\\-u_y \\ u_x \\ 0
 \end{pmatrix} +
(sin\theta)
\begin{pmatrix}
 u_x^2 \\ u_xu_y \\ u_xu_z\\
 u_yu_x \\ u_y^2 \\ u_yu_z\\
 u_zu_x \\ u_zu_y \\ u_z^2
 \end{pmatrix}
 \end{align}
 
 \begin{align}
J_{9\times3} = \begin{pmatrix}
\frac{\partial R}{\partial v_x} & \frac{\partial R}{\partial v_y} & \frac{\partial R}{\partial v_z}\\
\end{pmatrix}_{9\times3} = 
\begin{pmatrix}
\frac{\partial R}{\partial u_x} & \frac{\partial R}{\partial u_y} & \frac{\partial R}{\partial u_z} & \frac{\partial R}{\partial \theta}\\
\end{pmatrix}_{9\times4} \times B_{4\times4} \times A_{4\times3}
\end{align}


### Camera Model

OpenCv is using the following narrow field of view camera model. In this implementation we are focused on radial (3 parameters) and tangantial (2 parameters) distorion and ignoring other distortion parameters.

\begin{aligned}
\begin{pmatrix}
x_c\\
y_c\\
z_c\\
\end{pmatrix} =
\begin{pmatrix}
 r_{11} & r_{12} & r_{13} & t_{x}\\
 r_{21} & r_{22} & r_{23} & t_{y}\\
 r_{31} & r_{32} & r_{33} & t_{z}
 \end{pmatrix}
\begin{pmatrix}
x_w\\
y_w\\
z_w\\
1
\end{pmatrix}
\end{aligned}

\begin{aligned}
x'=  \frac{x_c}{y_c}\\
y'=  \frac{y_c}{z_c}\\
r = \sqrt{x'^2+y'^2} 
\end{aligned}

\begin{aligned}
x'' = x'(1 + k_1r^2 + k_2r^4 + k_3 r^6) + 2p_1x'y' + p_2(r^2 + 2x'^2)\\
y'' = y'(1 + k_1r^2 + k_2r^4 + k_3 r^6) + p_1(r^2 + 2y'^2) + 2p_2x'y' 
\end{aligned}

\begin{aligned}
\begin{pmatrix}
x_m\\
y_m
\end{pmatrix}=
 \begin{pmatrix}
 f_x & \alpha & c_x\\
 0 & f_y & c_y
 \end{pmatrix}
\begin{pmatrix}
x''\\
y''\\
1
\end{pmatrix}
\Rightarrow
\begin{cases}x_m=f_x x''+\alpha y''+ c_x  \\y_m=f_y y'' + c_y \end{cases} 
\end{aligned}

where $f_x$ and $f_y$ are focal length, $c_x$ and $c_y$ are image center, $\alpha$ is image skew, $k_1$, $k_2$ and $k_3$ are radial and $p_1$ and $P_2$ are tangential distortion parameters. 

\begin{aligned}
&\mathbf{p_w} =
\begin{pmatrix}
x_w\\
x_w\\
x_w\\
\end{pmatrix} , \space 
\mathbf{p_c} =
\begin{pmatrix}
x_c\\
y_c\\
z_c\\
\end{pmatrix}, \space
\mathbf{t} =
\begin{pmatrix}
t_x\\
t_y\\
t_z\\
\end{pmatrix}, \space
\mathbf{v} =
\begin{pmatrix}
v_x\\
v_y\\
v_z\\
\end{pmatrix}\\\\
&\mathbf{R}=Rodrigues(\mathbf{v}) = 
\begin{pmatrix}
r_{11} & r_{12} & r_{13}\\
r_{21} & r_{22} & r_{23}\\
r_{31} & r_{32} & r_{33}
\end{pmatrix}, \space
\mathbf{r_1} = \begin{pmatrix}
r_{11}\\r_{12} \\ r_{13}
\end{pmatrix}, \space
\mathbf{r_2} = \begin{pmatrix}
r_{21} \\ r_{22} \\ r_{23}
\end{pmatrix}, \space
\mathbf{r_3} = \begin{pmatrix}
r_{31} \\ r_{32} \\ r_{33}
\end{pmatrix}\\\\
&\mathbf{J_R} = 
\begin{pmatrix}
\frac{\partial \mathbf{r_1}}{\partial \mathbf{v}}\\
\frac{\partial \mathbf{r_2}}{\partial \mathbf{v}}\\
\frac{\partial \mathbf{r_3}}{\partial \mathbf{v}}\\
\end{pmatrix}, \space
\frac{\partial \mathbf{r_i}}{\partial \mathbf{v}}=
\begin{pmatrix}
\frac{\partial r_{i1}}{\partial v_x} & \frac{\partial r_{i1}}{\partial v_y} & \frac{\partial r_{i1}}{\partial v_z}\\
\frac{\partial r_{i2}}{\partial v_x} & \frac{\partial r_{i2}}{\partial v_y} & \frac{\partial r_{i2}}{\partial v_z}\\
\frac{\partial r_{i3}}{\partial v_x} & \frac{\partial r_{i3}}{\partial v_y} & \frac{\partial r_{i3}}{\partial v_z}
\end{pmatrix}
\end{aligned}


\begin{aligned}
x_c &= \mathbf{r_1}^T\mathbf{p_w} + t_x\\
y_c &= \mathbf{r_2}^T\mathbf{p_w} + t_y\\
z_c &= \mathbf{r_3}^T\mathbf{p_w} + t_z\\
\end{aligned}


We know if $\mathbf{y}$ is $n\times1$ and $\mathbf{x}$ is $n\times1$ and both $\mathbf{y}$ and $\mathbf{x}$ are functions of the vector $\mathbf{z}$, and $\mathbf{y^T}\mathbf{x}$ is an scalar, then \[Proposition 10, Ref. 7\]

\begin{aligned}
\frac{\partial (\mathbf{y^T}\mathbf{x})}{\partial \mathbf{z}}=
\mathbf{x^T}\frac{\partial \mathbf{y}}{\partial \mathbf{z}} + 
\mathbf{y^T}\frac{\partial \mathbf{x}}{\partial \mathbf{z}}
\end{aligned}

that results in

\begin{aligned}
\frac{\partial x_c}{\partial \mathbf{v}} = \mathbf{p_w^T}\frac{\partial \mathbf{r_1}}{\partial \mathbf{v}}\\\\
\frac{\partial y_c}{\partial \mathbf{v}} = \mathbf{p_w^T}\frac{\partial \mathbf{r_2}}{\partial \mathbf{v}}\\\\
\frac{\partial z_c}{\partial \mathbf{v}} = \mathbf{p_w^T}\frac{\partial \mathbf{r_3}}{\partial \mathbf{v}}
\end{aligned}

Also, we have

\begin{aligned}
\frac{\partial x_c}{\partial \mathbf{t}} = 
\begin{pmatrix}
1 & 0 & 0
\end{pmatrix}\\\\
\frac{\partial y_c}{\partial \mathbf{t}} = 
\begin{pmatrix}
0 & 1 & 0
\end{pmatrix}\\\\
\frac{\partial z_c}{\partial \mathbf{t}} = 
\begin{pmatrix}
0 & 0 & 1
\end{pmatrix}
\end{aligned}

$x' = x_c/z_c$ and $y' = y_c/z_c$ so we have:
\begin{aligned}
\frac{\partial x'}{\partial \mathbf{v}} = \frac{1}{z_c}\frac{\partial x_c}{\partial \mathbf{v}} - \frac{x_c}{z_c^2}\frac{\partial z_c}{\partial \mathbf{v}}=
\frac{1}{z_c}\left(\frac{\partial x_c}{\partial \mathbf{v}}-x'\frac{\partial z_c}{\partial \mathbf{v}} \right)\\\\
\frac{\partial y'}{\partial \mathbf{v}} = \frac{1}{z_c}\frac{\partial y_c}{\partial \mathbf{v}} - \frac{y_c}{z_c^2}\frac{\partial z_c}{\partial \mathbf{v}}=
\frac{1}{z_c}\left(\frac{\partial y_c}{\partial \mathbf{v}}-y'\frac{\partial z_c}{\partial \mathbf{v}} \right)
\end{aligned}

and similarly

\begin{aligned}
\frac{\partial x'}{\partial \mathbf{t}} = 
\frac{1}{z_c}\left(\frac{\partial x_c}{\partial \mathbf{t}}-x'\frac{\partial z_c}{\partial \mathbf{t}} \right)\\\\
\frac{\partial y'}{\partial \mathbf{t}} = 
\frac{1}{z_c}\left(\frac{\partial y_c}{\partial \mathbf{t}}-y'\frac{\partial z_c}{\partial \mathbf{t}} \right)
\end{aligned}

$r = \sqrt{x'^2+y'^2}$
\begin{aligned}
\frac{\partial r}{\partial \mathbf{v}} = \frac{x'}{\sqrt{x'^2+y'^2}}\frac{\partial x'}{\partial \mathbf{v}} + \frac{y'}{\sqrt{x'^2+y'^2}}\frac{\partial y'}{\partial \mathbf{v}} = \frac{1}{r}\left(x'\frac{\partial x'}{\partial \mathbf{v}} + y'\frac{\partial y'}{\partial \mathbf{v}}\right)
\end{aligned}

and similarly we have

\begin{aligned}
\frac{\partial r}{\partial \mathbf{t}} = \frac{1}{r}\left(x'\frac{\partial x'}{\partial \mathbf{t}} + y'\frac{\partial y'}{\partial \mathbf{t}}\right)
\end{aligned}


Ditortion parameters of the model $k_1, k_2, k_3, p_1, p_2$ come into the picture at this point:

\begin{aligned}
\mathbf{k} = \begin{pmatrix}
k_1 & k_2 & p_1 & p_2 & k_3
\end{pmatrix}
\end{aligned}

We define intermediate parameters $q_1, q_2, q_3, q_4$ as follows:

\begin{aligned}
q_1 = (1 + k_1r^2 + k_2r^4 + k_3 r^6)\\
q_2 = x'y'\\
q_3 = (r^2 + 2x'^2)\\
q_4 = (r^2 + 2y'^2)\\
\end{aligned}

\begin{aligned}
\frac{\partial q_1}{\partial \mathbf{k}}=\begin{pmatrix}r^2 & r^4 & 0 & 0 & r^6\end{pmatrix}\\
\frac{\partial q_1}{\partial \mathbf{v}}=
\begin{pmatrix}
2k_1r + 4k_2r^3 + 6k_3r^5
\end{pmatrix}
\frac{\partial r}{\partial \mathbf{v}}\\
\frac{\partial q_1}{\partial \mathbf{t}}=
\begin{pmatrix}
2k_1r + 4k_2r^3 + 6k_3r^5
\end{pmatrix}
\frac{\partial r}{\partial \mathbf{t}}
\end{aligned}

\begin{aligned}
\frac{\partial q_2}{\partial \mathbf{v}}= y'\frac{\partial x'}{\partial \mathbf{v}} + x'\frac{\partial y'}{\partial \mathbf{v}}\\
\frac{\partial q_2}{\partial \mathbf{t}}= y'\frac{\partial x'}{\partial \mathbf{t}} + x'\frac{\partial y'}{\partial \mathbf{t}}
\end{aligned}

\begin{aligned}
\frac{\partial q_3}{\partial \mathbf{v}}= 2r\frac{\partial r}{\partial \mathbf{v}} + 4x'\frac{\partial x'}{\partial \mathbf{v}}\\
\frac{\partial q_3}{\partial \mathbf{t}}= 2r\frac{\partial r}{\partial \mathbf{t}} + 4x'\frac{\partial x'}{\partial \mathbf{t}}
\end{aligned}

\begin{aligned}
\frac{\partial q_4}{\partial \mathbf{v}}= 2r\frac{\partial r}{\partial \mathbf{v}} + 4y'\frac{\partial y'}{\partial \mathbf{v}}\\
\frac{\partial q_4}{\partial \mathbf{t}}= 2r\frac{\partial r}{\partial \mathbf{t}} + 4y'\frac{\partial y'}{\partial \mathbf{t}}
\end{aligned}

We can rewrite $x'', y''$ as

\begin{aligned}
x'' = x'q_1 + 2p_1q_2 + p_2q_3\\
y'' = y'q_1 + p_1q_4 + 2p_2q_2 
\end{aligned}

\begin{aligned}
\frac{\partial x''}{\partial \mathbf{k}}=
x'\frac{\partial q_1}{\partial \mathbf{k}} + 
2q_2\begin{pmatrix}
0 & 0 & 1 & 0 & 0
\end{pmatrix} +
q_3\begin{pmatrix}
0 & 0 & 0 & 1 & 0
\end{pmatrix}
\end{aligned}

\begin{aligned}
\frac{\partial x''}{\partial \mathbf{v}}=
q_1\frac{\partial x'}{\partial \mathbf{v}} + 
x'\frac{\partial q_1}{\partial \mathbf{v}} +
2p_1\frac{\partial q_2}{\partial \mathbf{v}} +
p_2\frac{\partial q_3}{\partial \mathbf{v}}
\end{aligned}

\begin{aligned}
\frac{\partial x''}{\partial \mathbf{t}}=
q_1\frac{\partial x'}{\partial \mathbf{t}} + 
x'\frac{\partial q_1}{\partial \mathbf{t}} +
2p_1\frac{\partial q_2}{\partial \mathbf{t}} +
p_2\frac{\partial q_3}{\partial \mathbf{t}}
\end{aligned}

\begin{aligned}
\frac{\partial y''}{\partial \mathbf{k}}=
y'\frac{\partial q_1}{\partial \mathbf{k}} + 
q_4\begin{pmatrix}
0 & 0 & 1 & 0 & 0
\end{pmatrix} +
2q_2\begin{pmatrix}
0 & 0 & 0 & 1 & 0
\end{pmatrix}
\end{aligned}

\begin{aligned}
\frac{\partial y''}{\partial \mathbf{v}}=
q_1\frac{\partial y'}{\partial \mathbf{v}} + 
y'\frac{\partial q_1}{\partial \mathbf{v}} +
p_1\frac{\partial q_4}{\partial \mathbf{v}} +
2p_2\frac{\partial q_2}{\partial \mathbf{v}}
\end{aligned}

\begin{aligned}
\frac{\partial y''}{\partial \mathbf{t}}=
q_1\frac{\partial y'}{\partial \mathbf{t}} + 
y'\frac{\partial q_1}{\partial \mathbf{t}} +
p_1\frac{\partial q_4}{\partial \mathbf{t}} +
2p_2\frac{\partial q_2}{\partial \mathbf{t}}
\end{aligned}

Other parameters focal length, $f_x, f_y$, image center $c_x, c_y$, and skew $\alpha$ come to the picture at this point:
\begin{align}
x_m=f_x x''+\alpha y''+ c_x  \\y_m=f_y y'' + c_y
\end{align}

and final partial derivitives:

\begin{align}
\frac{\partial x_m}{\partial fx} = x'', \, \frac{\partial x_m}{\partial cx} = 1, \, \frac{\partial x_m}{\partial \alpha} = y''\\\\
\frac{\partial x_m}{\partial \mathbf{k}} = f_x\frac{\partial x''}{\partial \mathbf{k}}+
\alpha\frac{\partial y''}{\partial \mathbf{k}}\\\\
\frac{\partial x_m}{\partial \mathbf{v}} = f_x\frac{\partial x''}{\partial \mathbf{v}}+
\alpha\frac{\partial y''}{\partial \mathbf{v}}\\\\
\frac{\partial x_m}{\partial \mathbf{t}} = f_x\frac{\partial x''}{\partial \mathbf{t}}+
\alpha\frac{\partial y''}{\partial \mathbf{t}}\\
\end{align}

\begin{align}
\frac{\partial y_m}{\partial fy} = y'', \, \frac{\partial y_m}{\partial cy} = 1\\\\
\frac{\partial y_m}{\partial \mathbf{k}} = f_y\frac{\partial y''}{\partial \mathbf{k}}\\\\
\frac{\partial y_m}{\partial \mathbf{v}} = f_y\frac{\partial y''}{\partial \mathbf{v}}\\\\
\frac{\partial y_m}{\partial \mathbf{t}} = f_y\frac{\partial y''}{\partial \mathbf{t}}\\
\end{align}

The final Jacobian matrix can be constructed as two separate matrixes. One is matrix $J_{int}$ for intrinsic parameters as independent variables and the matrix $J_{ext}$ for extrinsic variables. The reason is that intrinsic parameters for all of the images are the same, but extrinsic parameters for each image is different and should be optimized independently. Jacobian matrices for a simgle point is constructed as follows. The matrices from all of $M$ points are stacked to construct final matrices for all of the points in one image.


\begin{align}
{J_{int}}_i = \begin{pmatrix}
\frac{\partial {x_m}_i}{\partial f_x} & \frac{\partial {x_m}_i}{\partial f_y}& \frac{\partial {x_m}_i}{\partial c_x} & \frac{\partial {x_m}_i}{\partial c_y} & \frac{\partial {x_m}_i}{\partial \alpha} & \frac{\partial {x_m}_i}{\partial k_1} & \frac{\partial {x_m}_i}{\partial k_2} & \frac{\partial {x_m}_i}{\partial p_1} & \frac{\partial {x_m}_i}{\partial p_2} & \frac{\partial {x_m}_i}{\partial k_3} \\
\frac{\partial {y_m}_i}{\partial f_x} & \frac{\partial {y_m}_i}{\partial f_y}& \frac{\partial {y_m}_i}{\partial c_x} & \frac{\partial {y_m}_i}{\partial c_y} & \frac{\partial {y_m}_i}{\partial \alpha} & \frac{\partial {y_m}_i}{\partial k_1} & \frac{\partial {y_m}_i}{\partial k_2} & \frac{\partial {y_m}_i}{\partial p_1} & \frac{\partial {y_m}_i}{\partial p_2} & \frac{\partial {y_m}_i}{\partial k_3} 
\end{pmatrix}\\\\
{J_{ext}}_i = \begin{pmatrix}
\frac{\partial {x_m}_i}{\partial v_x} & \frac{\partial {x_m}_i}{\partial v_y}& \frac{\partial {x_m}_i}{\partial v_z} & \frac{\partial {x_m}_i}{\partial t_x} & \frac{\partial {x_m}_i}{\partial t_y} & \frac{\partial {x_m}_i}{\partial t_z} \\
\frac{\partial {y_m}_i}{\partial v_x} & \frac{\partial {y_m}_i}{\partial v_y}& \frac{\partial {y_m}_i}{\partial v_z} & \frac{\partial {y_m}_i}{\partial t_x} & \frac{\partial {y_m}_i}{\partial t_y} & \frac{\partial {y_m}_i}{\partial t_z} \\
\end{pmatrix}\\
J_{int} =
\begin{pmatrix}
{J_{int}}_1\\
{J_{int}}_2\\
\vdots\\
{J_{int}}_M
\end{pmatrix}\,\,,\,\,
J_{ext} =
\begin{pmatrix}
{J_{ext}}_1\\
{J_{ext}}_2\\
\vdots\\
{J_{ext}}_M
\end{pmatrix}
\end{align}

## References

1. http://www.vision.caltech.edu/bouguetj/calib_doc/index.html#links
2. https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr98-71.pdf
3. https://www.slideshare.net/charmie11/bouguet-cameracalibrationtoolbox
4. Multiple View Geometry in Computer Vision, Hartley, R.~I. and Zisserman, A., 2004
5. https://en.wikipedia.org/wiki/Gauss%E2%80%93Newton_algorithm
6. https://en.wikipedia.org/wiki/Rotation_matrix
7. https://atmos.washington.edu/~dennis/MatrixCalculus.pdf